In [ ]:
import cv2
import pyttsx3
import numpy as np

def yolo():
# Load the pre-trained YOLOv3 model
        net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

        # Load the class names
        classes = []
        with open('coco.names', 'r') as f:
            classes = [line.strip() for line in f.readlines()]

        # Initialize the text-to-speech engine
        engine = pyttsx3.init()

        # Start capturing video from the webcam
        cap = cv2.VideoCapture(0)

        while True:
            # Read the current frame from the video stream
            ret, frame = cap.read()

            # Check if the frame is None (i.e., if the frame was not read successfully)
            if frame is None:
                print("Failed to read frame from video stream")
                break

            # Perform object detection on the frame
            height, width, channels = frame.shape
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(net.getUnconnectedOutLayersNames())

            # Get the bounding boxes, confidence scores, and class IDs
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        class_ids.append(class_id)
                        confidences.append(float(confidence))
                        boxes.append([x, y, w, h])

            # Apply non-maximum suppression to remove overlapping bounding boxes
            indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            # Loop over the remaining bounding boxes after non-maximum suppression
            for i in indices:
                if i<len(class_ids):
                    box = boxes[i]
                    left, top, width, height = box
                    label = f'{classes[class_ids[i]]}'


                # Convert the label to speech and speak it out
                    engine.say(label)
                    engine.runAndWait()

                # Print the label and confidence score in text format
                    print(label)

                # Draw the bounding box on the frame
                    cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)
            num_persons = label.count('person')


            # Display the resulting frame
            cv2.imshow('Object Detection', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        # Release the video capture and close the windows
        cap.release()
        cv2.destroyAllWindows()
yolo()